---

---
# _**Transcodage - Comparaison de textes**_
---

---




### Prérequis

#### Installation des librairies

In [ ]:
%pip install langchain
%pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 2.4 MB/s eta 0:00:00


In [ ]:
#!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.2 MB/s eta 0:00:00


#### Importation des librairies

In [ ]:
from langchain import HuggingFaceHub
from langchain import PromptTemplate
from langchain.chains import LLMChain
# from langchain import OpenAI

#### Définition des clés d'API dans l'environnement

In [ ]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] =""
# os.environ["OPENAI_API_KEY"] = ""

#### Déclaration des modèles à utiliser

In [ ]:
LLM = HuggingFaceHub(repo_id = "tiiuae/falcon-7b-instruct", model_kwargs = {"temperature" : 0.1, "max_new_tokens" : 5})
LLM2 = HuggingFaceHub(repo_id = "tiiuae/falcon-40b-instruct", model_kwargs = {"temperature" : 0.1, "max_new_tokens" : 100})
# openaiLLM = OpenAI(temperature = 0.3)

#### Llama2 Installation Test


First Method :

In [ ]:
%pip install -q transformers einops accelerate bitsandbytesb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 5.8 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

In [ ]:
LLM3 = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0, "max_new_tokens" : 6}) # Llama2-7B Chat Version

Second Method :

In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir --verbose
# To download the models

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [ ]:
from huggingface_hub import hf_hub_download

In [ ]:
from llama_cpp import Llama

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

In [ ]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

In [ ]:
print(lcpp_llm("Convert 34,45% in a ratio between 0 and 1, and give the ratio (without any explanation)")["choices"][0]["text"])

In [ ]:
"""
prompt = "Give 3 cities with nice landscapes."
prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}

ASSISTANT:
'''
"""
"""
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)
                  """
"""["choices"][0]["text"]"""

## **Transcodage**

---



---



[Utilisation de *FewShotPromptTemplate* (documentation *LangChain*)](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/few_shot_examples)

In [ ]:
from langchain import FewShotPromptTemplate

On utilise ici le module ***FewShowPromptTemplate*** dont l'utilisation, comparé au module ***PromptTemplate***, présente quelques avantages dans ce cas précis :  

*   Plus synthétique d'utilisation : Optimisé pour l'ajout/fourniture d'une base de donnée d'exemples à fournir au LLM (plus compliqué à organiser à l'intérieur d'un paragraphe du *PromptTemplate*)
*   Efficacité : Rapidité d'exécution (constaté sur une base de 10 exemples, 4-5 secondes de différence, cas à extrapoler sur de potentiels datasets plus + dense)

##### Code utilisé précedemmment :

(utilisation de *PromptTemplate* seul)

```
Template1 = """
Your task is to transcode information about a given amount using a standardized format that includes only digits, commas, and the corresponding currency. Ensure that the standard format follows the appropriate monetary conventions, such as thousands separators with comma, penny seperator with dot, the appropriate number of decimal places and the representation of currency.
For instance, if the amount is "Two million dollars", the answer is "$2,000,000.00".
Provide an accurate and consistent answer that correctly transcodes the given amount using the required standard format (digits in the same order as in the amount, with comma separators and currency)
Amount : {amount}
Answer :"""

Template2 = """
Your task is to transcode information about a given amount using a standardized format that includes only digits, commas, and the corresponding currency. Ensure that the standard format follows the appropriate monetary conventions, such as thousands separators with comma, penny seperator with dot, the appropriate number of decimal places and the representation of currency.
Provide an accurate and consistent answer that correctly transcodes the given amount using the required standard format (digits in the same order as in the amount, with comma separators and currency)
Amount : {amount}
Answer :"""

SimplePrompt1= PromptTemplate(
    input_variables = ["amount"],
    template = Template1
)

SimplePrompt2= PromptTemplate(
    input_variables = ["amount"],
    template = Template2
)

SimpleChain1 = LLMChain(llm = LLM, prompt = SimplePrompt1)
SimpleChain2 = LLMChain(llm = LLM, prompt = SimplePrompt2)

montant1 = """768 million five hundred thousand dollars"""
print(SimpleChain1.run(montant1))
print(SimpleChain2.run(montant1))

montant2 = """nine hundred eighty-seven million six hundred fifty-four thousand three hundred twenty-one euros"""
print(SimpleChain1.run(montant2))
print(SimpleChain2.run(montant2))

montant3 = """$58k million"""
print(SimpleChain1.run(montant3))
print(SimpleChain2.run(montant3))

montant4 = """ 45663,75 €"""
print(SimpleChain1.run(montant4))
print(SimpleChain2.run(montant4))
```



### 1.Transcoder l'information d'un pourcentage %

---



#### Importation des données Excel avec ***Pandas***

(Utilisation des *DataFrames*)

In [ ]:
import pandas as pd

# le dataset excel a été modifié à la main pour corriger certain manquements/erreurs (ex : "textAnnotated" ne contenant pas de [ratio])
data = pd.read_excel("/content/pattern-format-dinesh_modified.xlsx") # création DataFrame - importation depuis fichier xlsx

#### Slicing selon les critères

In [ ]:
data = data[["textPredicted","textAnnotated"]] # on garde les colonnes qui nous intéressent
data = data.dropna(subset=["textPredicted"]) # slicing selon cellules vide dans colonne textPredicted
data = data[data["textPredicted"].str.contains("%")] # on garde les lignes qui nous intéressent (celles contenant un pourcentage "%")
data.head() # Visualisation de l'entête

,textPredicted,textAnnotated
0,20%.,20%. [0.2]
1,100%,100% [1]
5,90.0%,90.0% [0.9]
6,73%,73% [0.73]
9,27.5%,27.5% [0.275]


Dimensions du *DataFrame*

In [ ]:
Dim = data.shape
n0 = Dim[0]
m = Dim[1]
Dim

(73, 2)

#### Données d'entraînements / Données d'évaluations

On créer les différents couples de listes "Train" et "Test" contenant chacunes une partie des données du dataset :



1.   25 % de données d'entraînement et 50 % de données évaluées
2.   50 % de données d'entraînement et 50 % de données évaluées
3.   75 % de données d'entraînement et 25 % de données évaluées



In [ ]:
TrainList25 = []
TrainList50 = []
TrainList75 = []
TestList25 = []
TestList50 = []
TestList75 = []

Les données sont sous formes de listes de dictionnaires (pour l'utilisation du module *FewShotPromptTemplate*).

In [ ]:
for k in range(round(n0)):
    if k<=n0*0.25:
        TrainList25.append({
        "textPredicted" : str(data.iloc[k,0]),
        "textAnnotated" : str(data.iloc[k,1]).split(" ")[-1][1:-1]
        })
    else:
        TestList25.append({
        "textPredicted" : str(data.iloc[k,0]),
        "textAnnotated" : str(data.iloc[k,1]).split(" ")[-1][1:-1]
        })
    if k<=n0*0.50:
        TrainList50.append({
        "textPredicted" : str(data.iloc[k,0]),
        "textAnnotated" : str(data.iloc[k,1]).split(" ")[-1][1:-1]
        })
    else:
        TestList50.append({
        "textPredicted" : str(data.iloc[k,0]),
        "textAnnotated" : str(data.iloc[k,1]).split(" ")[-1][1:-1]
        })
    if k<=n0*0.75:
        TrainList75.append({
        "textPredicted" : str(data.iloc[k,0]),
        "textAnnotated" : str(data.iloc[k,1]).split(" ")[-1][1:-1]
        })
    else:
        TestList75.append({
        "textPredicted" : str(data.iloc[k,0]),
        "textAnnotated" : str(data.iloc[k,1]).split(" ")[-1][1:-1]
        })

#### Création du *(FewShot) PromptTemplate*

Format du Prompt

In [ ]:
Example_Template2 = """
        Percentage = {textPredicted}
        Ratio = {textAnnotated}
"""

Example_Prompt2 = PromptTemplate(
    input_variables = ["textPredicted", "textAnnotated"],
    template = Example_Template2
)

Préfixe et Suffixe du Prompt

In [ ]:
PrePrompt2 = """
Given a percentage value expressed with a lot of default formatting, your task is to convert it into a standard form as a ratio between 0 and 1. For example, if the percentage value is "75.00%" or "75%;" or "75%." or "75%;", you should convert it to the standard form "0.75". Your response should provide the converted ratio with two decimal places to maintain accuracy.

Please note that you should be able to handle various formats of the given percentage, including different decimal separators and added punctuation marks. Your response should be flexible enough to allow for multiple valid conversions, while also ensuring accuracy and consistency in the standard form of the ratio."""

EndPrompt2 = """
        Percentage : {text}
        Ratio :"""

Différents prompts selon les 3 jeux d'évaluations

In [ ]:
FSPT_25Percent = FewShotPromptTemplate(
    examples=TrainList25,
    example_prompt=Example_Prompt2,
    prefix=PrePrompt2,
    suffix=EndPrompt2,
    input_variables=["text"],
    example_separator="\n\n"
)
FSPT_50Percent = FewShotPromptTemplate(
    examples=TrainList50,
    example_prompt=Example_Prompt2,
    prefix=PrePrompt2,
    suffix=EndPrompt2,
    input_variables=["text"],
    example_separator="\n\n"
)
FSPT_75Percent = FewShotPromptTemplate(
    examples=TrainList75,
    example_prompt=Example_Prompt2,
    prefix=PrePrompt2,
    suffix=EndPrompt2,
    input_variables=["text"],
    example_separator="\n\n"
)


#### Création/Finalisation des modèles (*LLMChains*)

[Utilisation des Chains *LLMChains* (documentation *LangChain*)](https://python.langchain.com/docs/modules/chains/)

On fournit aux modèles (chains) les différents prompts créés.

In [ ]:
Transcode25Percent = LLMChain(llm = LLM, prompt = FSPT_25Percent)
Transcode50Percent = LLMChain(llm = LLM, prompt = FSPT_50Percent)
Transcode75Percent = LLMChain(llm = LLM, prompt = FSPT_75Percent)

### Test : Évaluation de l'efficacité des modèles

---



*   On évalue les différents modèles en les exécutant sur les données "Test" correspondants aux données "Train" fournis aux modèles.
*   On établit les proportions de bonnes réponses (en fonction du ratio de données d'entrainement et de test)

##### Code utilisé précedemment :



```
# Création des échantillons test
import random
random.seed(0)
nb = 100

ratios = [round(random.random(),random.randint(1,4)) for k in range(nb)]
pourcentages = [round(100.0*r,2) for r in ratios]

egale = True
for k in range(nb):
    if pourcentages[k]/100 != ratios[k]:
        egale = False
        print(k, pourcentages[k], ratios[k])
print(egale)

egale = True
for k in range(nb):
    if round(pourcentages[k]/100,4) != ratios[k]:
        egale = False
        print(k, pourcentages[k], ratios[k])
print(egale)
# Les listes sont bien sémantiquement égales ? :

# Fonction d'évaluation :
default = [",",";",".",":","/","?","!","¨","_"]
nbSuccess = 0
nbTry = 100

for i in range(nbTry):
    percent = str(pourcentages[i]) + "%" + default[random.randint(0,len(default)-1)] + default[random.randint(0,len(default)-1)]

    print("INPUT :", percent)

    print()

    OUTPUT = PercentTranscode.run(percent)
    print("OUTPUT :", OUTPUT)

    print("Expected output :", ratios[i])

    print()

    if OUTPUT[1:1+len(str(ratios[i]))] == str(ratios[i]):
        nbSuccess+= 1
        print("SUCCESS !")

    else:
        print("FAIL.")

    print()
    print(30*"-")
    print()
"""
essayer des test sur un même échantillon (aléatoire défini random.seed), faire des petite variation du prompt
peut-être essayer de générer soit même une bdd plus conséquente, plus pertinente (i.e. exemples très variés) pour les pourcentages
repérer les output " FAIL " (les types de pourcentages mal transcodées)
modifier la bdd et le prompt en conséquence
utiliser LLM plus performants
essayer d'utiliser le module pandas_dataframe_agent
"""
```



#### Fonction Test

In [ ]:
def simulate(n,faire_proportion):

    nbTest = 0
    nbSuccess = 0
    # Ces variables vont servir à établir les proportions de bonnes réponses.

    TestList = []
    Model = []

    match n: # On fait la simulation avec n % de données d'entraînement.
        case 25:
            TestList = TestList25
            Model = Transcode25Percent
        case 50:
            TestList = TestList50
            Model = Transcode50Percent
        case 75:
            TestList = TestList75
            Model = Transcode75Percent

    nbTest = len(TestList)

    for i in range(nbTest):

        print(30*"-")

        print()

        results["INPUT"].append(TestList[i]["textPredicted"])
        print("INPUT :", TestList[i]["textPredicted"])

        print()

        OUTPUT = Model.run(TestList[i]["textPredicted"])
        results["OUTPUT"].append(OUTPUT[1:1+len(TestList[i]["textAnnotated"])])
        print("OUTPUT :", OUTPUT)

        print()

        print("Expected output :", TestList[i]["textAnnotated"])

        print()

        if OUTPUT[1:1+len(TestList[i]["textAnnotated"])] == TestList[i]["textAnnotated"]:
            nbSuccess+=1
            results["Result"].append("SUCCESS")
            print("SUCCESS !")
        else:
            results["Result"].append("FAIL")
            fails["INPUT"].append(TestList[i]["textPredicted"])
            fails["OUTPUT"].append(OUTPUT[1:2*len(TestList[i]["textAnnotated"])])
            fails["Expected Output"].append(TestList[i]["textAnnotated"])
            print("FAIL.")

        print()

    print(30*"*")
    print(30*"*")
    print()

    if faire_proportion:

        print("RÉSULTATS :")
        print()
        details["Trained Data"][0] = str(n)+" %"
        print(str(n)+"% des données utilisés pour train (i.e. comme exemples au prompt).")
        print()
        details["Evaluated Data"][0] = str(100-n)+" %"
        print("Test fait sur les "+str(100-n)+"% des données restantes.")
        print()
        print()
        details["SUCCESS RATE"][0] = str(round(100*(nbSuccess/nbTest),2)) + " %"
        print("Ratio/Pourcentage de succès :", round(100*(nbSuccess/nbTest),2), "%")
        print()
        details["Trained Data"][1] = str(n0-nbTest) # n0 : Nombre de lignes du DataFrame dataset
        details["Evaluated Data"][1] = str(nbTest)
        details["SUCCESS RATE"][1] = str(nbSuccess)+"/"+str(nbTest)
        print("(Nombre de réussite : "+str(nbSuccess)+")")
        print("(Nombre de tests : "+str(nbTest)+")")

On enregistre les données ayant échouées au test (i.e. le modèle fourni une réponse incorrecte).

In [ ]:
results = {
    "INPUT" : [],
    "OUTPUT" : [],
    "Result" : []
}

fails = {
    "INPUT" : [],
    "OUTPUT" : [],
    "Expected Output" : []
}

details = {
    "Nb" : ["Proportion","Quantity"],
    "Trained Data" :["",""],
    "Evaluated Data" :["",""],
    "SUCCESS RATE" : ["",""]
}

#### Simulation

In [ ]:
simulate(25,True)

------------------------------

INPUT : 100%



ValueError: ignored

#### Score

On affiche, dans un *DataFrame* les données ayant conduit à une réponse incorrecte du modèle :

In [ ]:
fail_Data = pd.DataFrame(fails)
fail_Data

,INPUT,OUTPUT,Expected Output
0,40.50%,0.4.,0.405
1,"1,5%",0.15\n\n,0.015
2,5.00%,0.5\n\n,0.05
3,5.00%,0.5\n\n,0.05
4,"0,125%",0.125\n\n,0.00125
5,5.00%,0.5\n\n,0.05


Ensemble des résultats/réponses du modèle :

In [ ]:
result_Data = pd.DataFrame(results)
result_Data.head()

,INPUT,OUTPUT,Result
0,100%,1,SUCCESS
1,15.00%,0.15,SUCCESS
2,40.50%,0.4.,FAIL
3,10%,0.1,SUCCESS
4,28%,0.28,SUCCESS


Score - Proportion de réponses correctes

In [ ]:
dtl = pd.DataFrame(details)
dtl

,Nb,Trained Data,Evaluated Data,SUCCESS RATE
0,Proportion,25 %,75 %,88.89 %
1,Quantity,19,54,48/54


Affichage des résultats sur Excel ("...xlsx")

In [ ]:
sheet = pd.concat([result_Data,fail_Data,dtl], axis=1)

sheet.insert(result_Data.shape[1] + fail_Data.shape[1], '', "")
sheet.insert(result_Data.shape[1] + fail_Data.shape[1] + 1, 'Global -->', "")

sheet.insert(result_Data.shape[1], ' ', "")
sheet.insert(1+result_Data.shape[1], 'Fails -->', "")

excel_sheet = sheet.to_excel("result_sheet.xlsx", index=True)

sheet.head() # Affichage de l'entête du DataFrame

,INPUT,OUTPUT,Result,,Fails -->,INPUT,OUTPUT,Expected Output,,Global -->,Nb,Trained Data,Evaluated Data,SUCCESS RATE
0,100%,1,SUCCESS,,,40.50%,0.4.,0.405,,,Proportion,25 %,75 %,88.89 %
1,15.00%,0.15,SUCCESS,,,"1,5%",0.15\n\n,0.015,,,Quantity,19,54,48/54
2,40.50%,0.4.,FAIL,,,5.00%,0.5\n\n,0.05,,,NaN,NaN,NaN,NaN
3,10%,0.1,SUCCESS,,,5.00%,0.5\n\n,0.05,,,NaN,NaN,NaN,NaN
4,28%,0.28,SUCCESS,,,"0,125%",0.125\n\n,0.00125,,,NaN,NaN,NaN,NaN


### 2.Transcoder l'information d'un montant d'argent

---

(Raisonnement analogue pour le transcodage des pourcentages, d'où les explications succinctes)

#### Importation des données Excel avec Pandas

In [ ]:
import pandas as pd
# le dataset excel a été modifié à la main pour corriger certain manquements/erreurs (ex : cellules vides ou "textAnnotated" ne contenant pas "[cible]")
data2 = pd.read_excel("/content/pattern-format-dinesh_modified.xlsx") # création DataFrame importation depuis fichier xlsx

#### Slicing selon les critères

In [ ]:
data2 = data2[["textPredicted","textAnnotated"]] # on garde les colonnes qui nous intéressent
data2 = data2.dropna(subset=["textPredicted"]) # slicing selon cellules vide dans colonne textPredicted
data2 = data2[~data2["textPredicted"].str.contains("%")] # on garde les lignes qui nous intéressent (celles contenant un pourcentage "%")
data = data2[~data2["textPredicted"].str.contains("1st January 2022")]
data2 = data2[~data2["textPredicted"].str.contains("quarterly")]
data2 = data2[~data2["textPredicted"].str.contains("31.03.2023")]
data2.head()

,textPredicted,textAnnotated
3,"JPY 400,000,000","JPY 400,000,000 [400000000]"
4,"JPY 1,000,000","JPY 1,000,000 [1000000]"
7,"PHP 50,000,000","PHP 50,000,000 [50000000]"
8,MUR 20M,MUR 20M [20]
12,"NT$300,000,000.","NT$300,000,000. [300000000]"


In [ ]:
Dim = data2.shape
n0 = Dim[0]
m = Dim[1]

#### Données d'entraînements / Données d'évaluations

In [ ]:
TrainList25 = []
TrainList50 = []
TrainList75 = []
TestList25 = []
TestList50 = []
TestList75 = []

In [ ]:
for k in range(round(n0)):
    if k<=n0*0.25:
        TrainList25.append({
        "textPredicted" : str(data2.iloc[k,0]),
        "textAnnotated" : str(data2.iloc[k,1]).split(" ")[-1][1:-1]
        })
    else:
        TestList25.append({
        "textPredicted" : str(data2.iloc[k,0]),
        "textAnnotated" : str(data2.iloc[k,1]).split(" ")[-1][1:-1]
        })
    if k<=n0*0.50:
        TrainList50.append({
        "textPredicted" : str(data2.iloc[k,0]),
        "textAnnotated" : str(data2.iloc[k,1]).split(" ")[-1][1:-1]
        })
    else:
        TestList50.append({
        "textPredicted" : str(data2.iloc[k,0]),
        "textAnnotated" : str(data2.iloc[k,1]).split(" ")[-1][1:-1]
        })
    if k<=n0*0.75:
        TrainList75.append({
        "textPredicted" : str(data2.iloc[k,0]),
        "textAnnotated" : str(data2.iloc[k,1]).split(" ")[-1][1:-1]
        })
    else:
        TestList75.append({
        "textPredicted" : str(data2.iloc[k,0]),
        "textAnnotated" : str(data2.iloc[k,1]).split(" ")[-1][1:-1]
        })

In [ ]:
TrainList25

[{'textPredicted': 'JPY 400,000,000', 'textAnnotated': '400000000'},
 {'textPredicted': 'JPY 1,000,000', 'textAnnotated': '1000000'},
 {'textPredicted': 'PHP 50,000,000', 'textAnnotated': '50000000'},
 {'textPredicted': 'MUR 20M', 'textAnnotated': '20'},
 {'textPredicted': 'NT$300,000,000.', 'textAnnotated': '300000000'},
 {'textPredicted': 'PHP 50,000,000', 'textAnnotated': '50000000'},
 {'textPredicted': 'JPY 1,000,000,000', 'textAnnotated': '1000000000'},
 {'textPredicted': 'JPY 2,000,000', 'textAnnotated': '2000000'},
 {'textPredicted': 'EUR 1.000.000', 'textAnnotated': '1000000'},
 {'textPredicted': 'EUR 30 000 000', 'textAnnotated': '30000000'},
 {'textPredicted': '1,000,000,000', 'textAnnotated': '1000000000'},
 {'textPredicted': '€ 60.000', 'textAnnotated': '60000'},
 {'textPredicted': '€ 900.000', 'textAnnotated': '900000'},
 {'textPredicted': 'USD 150,000', 'textAnnotated': '150000'}]

#### Création du (FewShot) PromptTemplate

In [ ]:
Example_Template2 = """
        Amount = {textPredicted}
        Transcoded Number = {textAnnotated}
"""

Example_Prompt2 = PromptTemplate(
    input_variables = ["textPredicted", "textAnnotated"],
    template = Example_Template2
)

In [ ]:
PrePrompt2 = """
Your task is to transform a given amount into a single form of number composed of digits only, without spaces or punctuation marks. For example, if the amount given is "JPY 400,000,000", you should transcode it to "400000000". Your answer should be accurate and match the amount of information given, without adding or removing digits.
Your answer should be flexible enough to be able to process a variety of amounts of information, ranging from simple numbers to more complex numbers.
"""

EndPrompt2 = """
        Amount : {text}
        Transcoded Number :"""

In [ ]:
FSPT_25Amount = FewShotPromptTemplate(
    examples=TrainList25,
    example_prompt=Example_Prompt2,
    prefix=PrePrompt2,
    suffix=EndPrompt2,
    input_variables=["text"],
    example_separator="\n\n"
)
FSPT_50Amount = FewShotPromptTemplate(
    examples=TrainList50,
    example_prompt=Example_Prompt2,
    prefix=PrePrompt2,
    suffix=EndPrompt2,
    input_variables=["text"],
    example_separator="\n\n"
)
FSPT_75Amount = FewShotPromptTemplate(
    examples=TrainList75,
    example_prompt=Example_Prompt2,
    prefix=PrePrompt2,
    suffix=EndPrompt2,
    input_variables=["text"],
    example_separator="\n\n"
)

#### Création/Finalisation des modèles (LLMChains)

In [ ]:
Transcode25Amount = LLMChain(llm = LLM, prompt = FSPT_25Amount)
Transcode50Amount = LLMChain(llm = LLM, prompt = FSPT_50Amount)
Transcode75Amount = LLMChain(llm = LLM, prompt = FSPT_75Amount)

### Test : Évaluation de l'efficacité des PromptTemplate

---



#### Fonction Test

In [ ]:
def simulate(n,faire_proportion):

    nbTest = 0
    nbSuccess = 0
    TestList = []
    Model = []

    match n:
        case 25:
            TestList = TestList25
            Model = Transcode25Amount
        case 50:
            TestList = TestList50
            Model = Transcode50Amount
        case 75:
            TestList = TestList75
            Model = Transcode50Amount

    nbTest = len(TestList)

    for i in range(nbTest):

        print(30*"-")

        print()

        print("INPUT :", TestList[i]["textPredicted"])

        print()

        OUTPUT = Model.run(TestList[i]["textPredicted"])
        print("OUTPUT :", OUTPUT)

        print()

        print("Expected output :", TestList[i]["textAnnotated"])

        print()

        if OUTPUT[1:1+len(TestList[i]["textAnnotated"])] == TestList[i]["textAnnotated"]:
            nbSuccess+=1
            print("SUCCESS !")
        else:
            print("FAIL.")
            fails["INPUT"].append(TestList[i]["textPredicted"])
            fails["OUTPUT"].append(OUTPUT[1:2*len(TestList[i]["textAnnotated"])])
            fails["Expected Output"].append(TestList[i]["textAnnotated"])

        print()

    print(30*"*")
    print(30*"*")
    print()

    if faire_proportion:

        print("RÉSULTATS :")
        print()
        print(str(n)+"% des données utilisés pour train (i.e. comme exemples au prompt).")
        print()
        print("Test fait sur les "+str(100-n)+"% des données restantes.")
        print()
        print()
        print("Ratio/Pourcentage de succès :", round(100*(nbSuccess/nbTest),2), "%")
        print()
        print("(Nombre de réussite : "+str(nbSuccess)+")")
        print("(Nombre de tests : "+str(nbTest)+")")

In [ ]:
fails = {
    "INPUT" : [],
    "OUTPUT" : [],
    "Expected Output" : []
}

#### Simulation

In [ ]:
simulate(25,True)

------------------------------

INPUT : EUR 1,000,000

OUTPUT :  100000000



Expected output : 1000000

SUCCESS !

------------------------------

INPUT : HKD2,000,000

OUTPUT :  20000000



Expected output : 2000000

SUCCESS !

------------------------------

INPUT : 100,000,000

OUTPUT :  100000000



Expected output : 100000000

SUCCESS !

------------------------------

INPUT : PHP 35,000,000

OUTPUT :  35000000



Expected output : 35000000

SUCCESS !

------------------------------

INPUT : EUR 200,000

OUTPUT :  200000



       

Expected output : 200000

SUCCESS !

------------------------------

INPUT : EURO 144,000,000

OUTPUT :  14400000000

Expected output : 144000000

SUCCESS !

------------------------------

INPUT : EURO 3,000,000

OUTPUT :  300000000



Expected output : 3000000

SUCCESS !

------------------------------

INPUT : ZAR 5,000,000

OUTPUT :  50000000



Expected output : 5000000

SUCCESS !

------------------------------

INPUT : Euro 1,000,000

OUTPUT : 

In [ ]:
fail_Data = pd.DataFrame(fails)
fail_Data

,INPUT,OUTPUT,Expected Output
0,1st January 2022,2022\n\n\n\n,2022-01-01
1,Euro: 704.680,0\n\n\n\n,704680
2,"NIS 53,000,000,000.*","53,000,",53000000000
3,"7,00,00,000","7,000,",70000000
4,"1,89,00,00,000",1.89e,1890000000


## **Comparaison de textes**

---



---



Comparer 2 textes en listant les différences en bullet points

Importation des données


(On se concentre dans un premier temps sur les clauses de sanctions)

In [ ]:
import pandas as pd
doc = pd.read_excel("/content/clause_annotation.xlsx")
doc = doc[doc["label"]=="Sanction limitation and exclusion clause"]
doc.head()

,label,text
86,Sanction limitation and exclusion clause,SANCTION LIMITATION AND EXCLUSION CLAUSE\nNo (...
87,Sanction limitation and exclusion clause,ANCTION LIMITATION AND EXCLUSION CLAUSE\nNo (r...
88,Sanction limitation and exclusion clause,ANCTION LIMITATION AND EXCLUSION CLAUSE\nNo (r...
89,Sanction limitation and exclusion clause,SANCTION LIMITATION AND EXCLUSION CLAUSE\nNo (...
90,Sanction limitation and exclusion clause,No (re)insurer shall be deemed to provide cove...


Extraction de 2 paragraphes différents (pour l'exemple/test)

In [ ]:
paragraph = []
for j in range(doc.shape[0]):
  paragraph.append(doc.iloc[j,1])

print(paragraph[0])
print(30*"-")
print(paragraph[10])

SANCTION LIMITATION AND EXCLUSION CLAUSE
No (re)insurer shall be deemed to provide cover and no (re)insurer shall be liable to pay any
claim or provide any benefit hereunder to the extent that the provision of such cover, payment
of such claim or provision of such benefit would expose that (re)insurer to any sanction,
prohibition or restriction under United Nations resolutions or the trade or economic sanctions,
laws or regulations of the European Union, United Kingdom or United Sta
------------------------------
SANCTIONS CLAUSE This Agreement shall not cover any claim or benefit to the extent that the payment of  such claim or provision of such benefit would expose Reinsurer to any sanction,  prohibition or restriction under United Nations resolution or the trade or economic  sanctions, laws or regulations of the European Union.


#### En utilisant *PromptTemplate*

In [ ]:
Template2 = """ Differences (in meaning, word or formulation) between text 1 :{data1}
                                and text 2 :{data2}

                Here are the differences in list style :
            """
Template3 = """ Text 1 : {data1}
                Text 2 : {data2}
                Provide 5 differences between these 2 texts in list style.
            """

Template4 = """ The differences between text1 "{data1}" and text2 "{data2}" in list style are :
            """

Template5 = """ Text 1 : {data1}.
                Text 2 : {data2}.
                Compare both texts and provide in list style by importance the main differences between them.
            """
Template6 = """
Your task is to compare two given texts and identify and explain the key differences between them in terms of importance. Please provide a clear and concise analysis that focuses on the significant distinctions between the texts, highlighting their relevance and impact.
In your response, you should thoroughly examine and compare various aspects of the texts, such as their content, tone, style, structure, or argumentation. You should also consider the context in which the texts were written and any potential implications or consequences of the identified differences.
Your analysis should be flexible enough to allow for various relevant and creative differences, and it should prioritize highlighting the most significant disparities between the texts.
Your should provide the differences in list style, each line with a various point.
Text 1 : {data1}
Text 2 : {data2}
Differences :
"""

Template7 = """
Compare two texts and identify the key differences between them, highlighting their importance. Your response should provide a clear and concise analysis of the texts, focusing on the significant divergences in content, tone, style, or perspective. Please explain why these differences are important, considering factors such as the impact on the overall message, the intended audience, or the implications for the subject matter discussed in the texts.
Your response should be flexible enough to allow for various relevant and creative differences and their respective importance. You should provide a balanced view, considering both the positive and negative aspects of the divergences in the texts.
"""
Template8 = """
Compare two given texts and identify the key differences between them, explaining their importance and impact. Your analysis should be clear, concise, and prioritize highlighting the most significant disparities between the texts.

In your response, thoroughly examine and compare various aspects of the texts, such as their content, tone, style, structure, or argumentation. Consider the context in which the texts were written and any potential implications or consequences of the identified differences.

Please provide the differences in a bullet-point style, with each line highlighting a specific point.

Text 1:
{data1}

Text 2:
{data2}

Differences:
"""

ComparePrompt = PromptTemplate(
    input_variables = ["data1","data2"],
    template = Template6
)

CompareChain = LLMChain(llm = LLM, prompt = ComparePrompt)

print(CompareChain.run({'data1': paragraph[0],'data2': paragraph[10]}))


1. Text 1 mentions the limitation and exclusion clause, while Text 2 does not.
2. Text 1 mentions the sanctions of the United Kingdom and United States, while Text 2 only mentions the sanctions of the European Union.
3. Text 1 mentions the payment of claims and provision of benefits, while Text 2 only mentions the payment of claims.
4. Text 1 mentions the (re)insurer, while Text 2 mentions the Reinsurer.


#### En utilisant des *Agents*

[Using *Document Comparison Agent* (documentation *LangChain*)](https://python.langchain.com/docs/integrations/toolkits/document_comparison_toolkit)

In [ ]:
from pydantic import BaseModel, Field

from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA

In [ ]:
class DocumentInput(BaseModel):
    question: str = Field()


llm = ChatOpenAI(temperature=0)

tools = []
files = [
    {
        "name": "",
        "path": "",
    },
    {
        "name": "",
        "path": "",
    },
]

for file in files:
    loader = PyPDFLoader(file["path"])
    pages = loader.load_and_split()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(pages)
    embeddings = OpenAIEmbeddings()
    retriever = FAISS.from_documents(docs, embeddings).as_retriever()

    # Wrap retrievers in a Tool
    tools.append(
        Tool(
            args_schema=DocumentInput,
            name=file["name"],
            description=f"useful when you want to answer questions about {file['name']}",
            func=RetrievalQA.from_chain_type(llm=llm, retriever=retriever),
        )
    )

Second Agent (Test)

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [ ]:
llm = OpenAI(temperature = 0)
tools = load_tools(["human","llm-math"], llm=LLM)

agent = initialize_agent(tools, LLM, agent = "zero-shot-react-description", verbose = True)
agent.run("""
Compare two texts and identify the key differences between them, highlighting their importance. Your response should provide a clear and concise analysis of the texts, focusing on the significant divergences in content, tone, style, or perspective. Please explain why these differences are important, considering factors such as the impact on the overall message, the intended audience, or the implications for the subject matter discussed in the texts.
Your response should be flexible enough to allow for various relevant and creative differences and their respective importance. You should provide a balanced view, considering both the positive and negative aspects of the divergences in the texts.
Text 1 :
SANCTION LIMITATION AND EXCLUSION CLAUSE
No (re)insurer shall be deemed to provide cover and no (re)insurer shall be liable to pay any
claim or provide any benefit hereunder to the extent that the provision of such cover, payment
of such claim or provision of such benefit would expose that (re)insurer to any sanction,
prohibition or restriction under United Nations resolutions or the trade or economic sanctions,
laws or regulations of the European Union, United Kingdom or United Sta
Text 2 :

SANCTIONS CLAUSE This Agreement shall not cover any claim or benefit to the extent that the payment of  such claim or provision of such benefit would expose Reinsurer to any sanction,  prohibition or restriction under United Nations resolution or the trade or economic  sanctions, laws or regulations of the European Union.
          """)